In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp meiosis

In [ ]:
from chewc.core import *
import torch

## meiosis
> Simulating Meisois and Recombination for various Crossing actions

In [ ]:
#| export
from chewc.core import *
import torch
def gamma_interference_model(length, rate, shape):
    """
    Simulate crossover events using a gamma interference model.
    
    Parameters:
    length (float): Length of the chromosome.
    rate (float): Rate of crossover events.
    shape (float): Shape parameter for the gamma distribution.
    
    Returns:
    torch.Tensor: Positions of crossover events.
    """
    num_crossovers = torch.poisson(torch.tensor([rate * length]))
    intervals = torch.distributions.gamma.Gamma(shape, rate).sample((int(num_crossovers.item()),))
    crossover_positions = torch.cumsum(intervals, dim=0)
    crossover_positions = crossover_positions[crossover_positions < length]
    return crossover_positions


def simulate_meiosis(num_chromosomes, chromosome_length, crossover_rate, interference_strength):
    """
    Simulate meiosis with crossover events and interference.
    
    Parameters:
    num_chromosomes (int): Number of chromatids.
    chromosome_length (float): Length of the chromosome.
    crossover_rate (float): Rate of crossover events.
    interference_strength (float): Strength of interference.
    
    Returns:
    list of torch.Tensor: List of crossover positions for each chromatid.
    """
    chromatid_crossovers = []
    for _ in range(num_chromosomes):
        crossovers = gamma_interference_model(chromosome_length, crossover_rate, interference_strength)
        chromatid_crossovers.append(crossovers)
    return chromatid_crossovers

def simulate_gametes(genetic_map, parent_genome):
    """
    Simulate the formation of a single gamete given crossover positions, genetic map, and parent genomes.

    Parameters:
    crossover_positions (list of torch.Tensor): List of positions of crossover events for each chromosome.
    genetic_map (list of torch.Tensor): List of positions of genetic markers on the chromosomes.
    parent_genome (torch.Tensor): Genomes of the parents. SHAPE:  (ploidy, num_chromosomes, num_loci)

    Returns:
    torch.Tensor: The resultant single gamete. SHAPE: (ploidy//2, num_chromosomes, num_loci)
    """
    ploidy, num_chromosomes, num_loci = parent_genome.shape
    gamete_genome = torch.zeros((ploidy // 2, num_chromosomes, num_loci), dtype=parent_genome.dtype)
    crossover_positions = simulate_meiosis(num_chromosomes,100.0,1,1)
    for chrom in range(num_chromosomes):
        parent_1 = parent_genome[0, chrom]
        parent_2 = parent_genome[1, chrom]
        
        crossover_sites = crossover_positions[chrom]
        chromatid = parent_1.clone()
        
        if len(crossover_sites) > 0:
            crossover_index = 0
            for i, marker_position in enumerate(genetic_map[chrom]):
                if crossover_index < len(crossover_sites) and marker_position >= crossover_sites[crossover_index]:
                    chromatid = parent_2 if chromatid is parent_1 else parent_1
                    crossover_index += 1
                gamete_genome[0, chrom, i] = chromatid[i]

    return gamete_genome


In [ ]:
ploidy = 2
number_chromosomes = 10
loci_per_chromosome = 100
n_founders = 50
genetic_map = create_random_genetic_map(number_chromosomes,loci_per_chromosome)
crop_genome = Genome(ploidy, number_chromosomes, loci_per_chromosome, genetic_map)
founder_pop = create_random_founder_pop(crop_genome , n_founders)

In [ ]:
simulate_gametes(genetic_map, founder_pop[0]).shape

torch.Size([1, 10, 100])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()